In [27]:
import pytesseract
from PIL import Image
import pandas as pd
import cv2 as cv
from pdf2image import convert_from_path 
import os

In [28]:
# function to do skew correction in images
def deskew(img):
    img = cv2.Canny(img, 100, 200)
    coords = np.column_stack(np.where(img > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = img.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

In [29]:
# function to do image processing on the image
def img_processing(image):
    img = cv2.resize(image, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv.COLOR_BGR2GRAY)
    #kernel = np.ones((1, 1), np.uint8)
    #img = cv2.dilate(img, kernel, iterations=1)
    #img = cv2.erode(img, kernel, iterations=1)
    img = cv.GaussianBlur(img, (9, 9), 0)
    #img = cv.bilateralFilter(img,9,75,75)
    #img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    return img

In [30]:
# function to find text in image
def ocr(image):
    text = pytesseract.image_to_string(image)
    return text

In [31]:
# function to check whether GSTIN is present or not in image
def check_GSTIN(text):
    flag1 = text.find('GSTIN')
    if(flag1 > 0):
        return flag1 +  6
    
    flag2 = text.find('STIN')
    if(flag2 > 0):
        return flag2  + 5
    
    return -1

In [32]:
# function to find GST Number in image
def GST_number(text,pos):
    count = 0
    number = []
    while((text[pos+count].isdigit() == False) and count <10):
        count+=1
    if(text[pos+count].isdigit()):
        if(text[pos+count-1].isalpha()):
            count = count - 1
            number = text[pos+count:pos+count+15]
            
    return number

In [ ]:
path = "data/"
dirs = os.listdir( path )
for i in dirs:
    print(os.path.join(path,i))
    filename, file_extension = os.path.splitext(os.path.join(path,i))
    if (file_extension=='.pdf'):
        pages = convert_from_path(os.path.join(path,i),  fmt='jpg') 
        print(pages)
    elif (file_extension=='.jpg'):
        img = cv.imread(filename+file_extension)
    
    


    
    #     cv.imshow('Image',img)
#     cv.waitKey(0)
#     cv.destroyAllWindows()
    # Do processing on image
    #processed_img = img_processing(image)
    
    # Find texts on image
    text = ocr(img)
    print(text)
    
    # Check GSTIN is present or not
#     flag_GSTIN = check_GSTIN(text)
#     if(flag_GSTIN>=0):
#         GSTIN_detected += 1
#         data.iloc[i]['Label'] = 'GSTIN_Detected'
#     else:
#         data.iloc[i]['Label'] = 'GSTIN_Not_Detected'
    
    # If GSTIN detected find the number
    #if(flag_GSTIN>=0):
    #    gst_number = GST_number(text,flag_GSTIN)
    #    data.iloc[i]['GST_Number'] = gst_number
    #else:
    #    data.iloc[i]['GST_Number'] = None
    
    # Display Image
    cv.imshow("image",img)
#     cv.imshow('After Processing',processed_img)
    cv.waitKey(0)
    cv.destroyAllWindows()

# print(GSTIN_detected)

data/invoices (1).jpg
INVOICE Ne

  

 

$2000
data/invoices (8).jpg
ZYLKAR CORPORATION
1561 Appleview Town
Bakers Street

Chicago, IL 60411

Bill to: Ship To:

Alistair Burke Angeline Campbell

1156 High Street 6040 Bandini Blvd

Santa Cruz, CA 95064 Los Angeles, CA 92061

800-555-0121 888-555-2235

ID Product Description Qty

1 Ream of Paper 5

2 Desk 5

3 Windows Workstation 5

Thank you for your business ! Subtotal
Tax Rate
Freight
Total

Signature / Stamp

Place

Date

PRO FORMA INVOICE
Invoice: #00121

July 21,2015
Shipping Details:
Freight Type Air
Est Ship Date July 27, 2015

Est Gross Weight 300 kg
Total Packages 10

Unit Price Amount (in USD)
$50 $250.00

$100 $500.00
$1000 $5000.00
$5750.00

5.00%

$100.00

 

Powered by @REXEE! Invoice
